## Data Preprocessing Simple

In [1]:
import pandas
import numpy as np
import seaborn as sns
import sklearn
from sklearn.impute import SimpleImputer
import sklearn.impute
import sklearn.preprocessing
import matplotlib.pyplot as plt
import missingno as msno

In [2]:
data = pandas.read_csv('/Users/tanananyathongkum/Documents/Visual Studio Code/Project_CE/data/heart_disease.csv')
data

,Age,Gender,Blood Pressure,Cholesterol Level,Exercise Habits,Smoking,Family Heart Disease,Diabetes,BMI,High Blood Pressure,...,High LDL Cholesterol,Alcohol Consumption,Stress Level,Sleep Hours,Sugar Consumption,Triglyceride Level,Fasting Blood Sugar,CRP Level,Homocysteine Level,Heart Disease Status
0,56.0,Male,153.0,155.0,High,Yes,Yes,No,24.991591,Yes,...,No,High,Medium,7.633228,Medium,342.0,NaN,12.969246,12.387250,No
1,69.0,Female,146.0,286.0,High,No,Yes,Yes,25.221799,No,...,No,Medium,High,8.744034,Medium,133.0,157.0,9.355389,19.298875,No
2,46.0,Male,126.0,216.0,Low,No,No,No,29.855447,No,...,Yes,Low,Low,4.440440,Low,393.0,92.0,12.709873,11.230926,No
3,32.0,Female,122.0,293.0,High,Yes,Yes,No,24.130477,Yes,...,Yes,Low,High,5.249405,High,293.0,94.0,12.509046,5.961958,No
4,60.0,Male,166.0,242.0,Low,Yes,Yes,Yes,20.486289,Yes,...,No,Low,High,7.030971,High,263.0,154.0,10.381259,8.153887,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,25.0,Female,136.0,243.0,Medium,Yes,No,No,18.788791,Yes,...,Yes,Medium,High,6.834954,Medium,343.0,133.0,3.588814,19.132004,Yes
9996,38.0,Male,172.0,154.0,Medium,No,No,No,31.856801,Yes,...,Yes,NaN,High,8.247784,Low,377.0,83.0,2.658267,9.715709,Yes
9997,73.0,Male,152.0,201.0,High,Yes,No,Yes,26.899911,No,...,Yes,NaN,Low,4.436762,Low,248.0,88.0,4.408867,9.492429,Yes
9998,23.0,Male,142.0,299.0,Low,Yes,No,Yes,34.964026,Yes,...,Yes,Medium,High,8.526329,Medium,113.0,153.0,7.215634,11.873486,Yes


In [ ]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns
categorical_features

In [ ]:
data[categorical_features].isna().sum(axis='index')

In [ ]:
categorical_imputer = sklearn.impute.SimpleImputer(strategy='most_frequent')
data[categorical_features] = categorical_imputer.fit_transform(data[categorical_features])

In [ ]:
data[categorical_features].isna().sum(axis='index')

In [ ]:
onehot_features = [
    'Gender', 
    'Smoking', 
    'Family Heart Disease', 
    'Diabetes', 
    'High Blood Pressure', 
    'Low HDL Cholesterol', 
    'High LDL Cholesterol', 
    
    ]

onehot_mapping = [
    ('Male', 'Female'),
    ('No', 'Yes'),
    ('No', 'Yes'),
    ('No', 'Yes'),
    ('No', 'Yes'),
    ('No', 'Yes'),
    ('No', 'Yes'),
   
]

ordinal_features = [
    'Exercise Habits',
    'Alcohol Consumption',
    'Stress Level',
    'Sugar Consumption'
]

ordinal_mapping = [
     ('Low', 'Medium', 'High'),
    ('Low', 'Medium', 'High'),
    ('Low', 'Medium', 'High'),
    ('Low', 'Medium', 'High')
]

mapping = {
    'Heart Disease Status': {
        'No': 0,
        'Yes': 1
    }
}

In [ ]:
onehot_encoder = sklearn.preprocessing.OneHotEncoder(categories=onehot_mapping, sparse_output=False)
onehot_encoder

In [ ]:
onehot_encoder.fit(data[onehot_features])

In [ ]:
onehot_encoder.get_feature_names_out()

In [ ]:
data_onehot_features = pandas.DataFrame(
        data = onehot_encoder.transform(data[onehot_features]),
        columns = onehot_encoder.get_feature_names_out(),
        index = data.index  
    )
data_onehot_features

In [ ]:
ordinal_encoder = sklearn.preprocessing.OrdinalEncoder(categories=ordinal_mapping)

In [ ]:
ordinal_encoder.fit(data[ordinal_features])

In [ ]:
ordinal_encoder.get_feature_names_out()

In [ ]:
data_ordinal_features = pandas.DataFrame(
        data = ordinal_encoder.transform(data[ordinal_features]),
        columns = ordinal_encoder.get_feature_names_out(),
        index = data.index  
    )
data_ordinal_features

In [ ]:
categorical_features = (
    data_ordinal_features
    .join(data_onehot_features)
)
categorical_features

In [ ]:
numeric_features = data.select_dtypes(include=['number']).columns
def cap_outliers(data, lower_percentile=0.01, upper_percentile=0.99):
    df_copy = data.copy()
    for col in numeric_features:
        lower_bound = df_copy[col].quantile(lower_percentile)
        upper_bound = df_copy[col].quantile(upper_percentile)
        df_copy[col] = df_copy[col].clip(lower=lower_bound, upper=upper_bound)
    return df_copy

numeric_features = cap_outliers(data[numeric_features])
numeric_features

In [ ]:
plt.figure(figsize=(12, 0.6*len(numeric_features.columns)+2))
sns.boxplot(data=numeric_features, orient='h', showfliers=True)
plt.title("Boxplots (outliers)")
plt.tight_layout()
plt.show()

In [ ]:
numeric_features.describe()

In [ ]:
numeric_imputer = sklearn.impute.SimpleImputer(strategy='mean')
data[numeric_features] = numeric_imputer.fit_transform(data[numeric_features])

In [ ]:
data[categorical_features].isna().sum(axis='index')

In [ ]:
MinMaxScaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True)
MinMaxScaler

In [ ]:
MinMaxScaler.fit(numeric_features)

In [ ]:
numeric_features = pandas.DataFrame(
        data = MinMaxScaler.transform(numeric_features),
        columns = numeric_features.columns,
        index = numeric_features.index
    )
numeric_features

In [ ]:
mapping = pandas.DataFrame(
    data = data.replace(mapping),
    columns = ['Heart Disease Status'],
    index = data.index
)
mapping

In [ ]:
data =(
    numeric_features
    .join(categorical_features)
    .join(mapping)
)
data

In [ ]:
data.to_csv('/Users/tanananyathongkum/Documents/Visual Studio Code/Project_CE/data/heart_disease_clean_mean.csv', index=False)
